In [1]:
import requests, json, pandas as pd

In [2]:
def marketsMojoPeers(mm_stockId):
    """ 
    Returns the Quality, Valuation and Financial Trend with Short Term Technical Rating from Markets Mojo for all its peers
    
    Input--> Markets Mojo Stock ID  (Example mm_stockId - 592009 - HDFC BANK)
    
    Output --> Pandas Dataframe with Mojo Metrics
    
    
    """
    #import pandas as pd, requests, json
    url = "https://frapi.marketsmojo.com/stocks/getPeersDetails"

    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0',
      'Accept': 'application/json, text/plain, */*',
      'Accept-Language': 'en-US,en;q=0.5',
      'Content-Type': 'text/plain',
      'Origin': 'https://www.marketsmojo.com',
      'DNT': '1',
      'Connection': 'keep-alive',
      'Referer': 'https://www.marketsmojo.com/',
      'TE': 'Trailers',
      'Pragma': 'no-cache',
      'Cache-Control': 'no-cache'
    }
    try:
        payload = '{\"stock_id\":\"'+ mm_stockId + '\",\"popup\":1}'
        
        response = requests.request("POST", url, headers=headers, data = payload)
        
        stock_list = {}
        
        for each_stock in response.json()['data']['peer']['list']: 
                stock = {}
                stock['Name'] = each_stock['details']['sname']
                stock['Industry'] = each_stock['details']['ind_name']
                stock['quality'] = each_stock['dot']['q_txt']
                stock['valuation'] = each_stock['dot']['v_txt']
                stock['financial'] = each_stock['dot']['f_txt']
                stock['technical'] = each_stock['dot']['f_txt']
                stock['Mojo'] = each_stock['dot']['scoreText']
                stock_list[each_stock['details']['sid']] = stock
                
        mojo_temp = pd.DataFrame.from_dict(stock_list).transpose()
        
        return mojo_temp
    
    except:
        print('Unable to find the Stock with Markets Mojo Stock ID %s' %(mm_stockId)  )


In [3]:
mojo = pd.read_csv("MojoWithCodes.csv")
mmid = mojo.groupby('Industry').first()['mm_id']
mojo_temp = marketsMojoPeers('643071')

for industry, stock_id in mmid.iteritems():
    print(f'Getting Peer Group for {industry}')
    mojo_temp = mojo_temp.append(marketsMojoPeers(str(stock_id)))
mojo = pd.read_csv("MojoWithCodes.csv", index_col='mm_id')    
mojo_temp = mojo[['BSEID', 'NSEID']].join(mojo_temp)

Getting Peer Group for Agri-Seeds
Getting Peer Group for Air Conditioners
Getting Peer Group for Airlines
Getting Peer Group for Aluminium
Getting Peer Group for Auto - 2 &3 Wheel
Getting Peer Group for Auto - Cars
Getting Peer Group for Auto - Tractor
Getting Peer Group for Auto - Trucks
Getting Peer Group for Auto Anc
Getting Peer Group for BPO/ITeS
Getting Peer Group for Batteries
Getting Peer Group for Breweries
Getting Peer Group for Cable
Getting Peer Group for Capital Goods
Getting Peer Group for Carbon Black
Getting Peer Group for Cement
Getting Peer Group for Ceramics and Allied
Getting Peer Group for Chemicals
Getting Peer Group for Cigarettes/Tobacco
Getting Peer Group for Compressors / Pumps
Getting Peer Group for Construction Material
Getting Peer Group for Consumer Electronics
Getting Peer Group for Consumer durables
Getting Peer Group for DTH/Cable
Getting Peer Group for Defence
Getting Peer Group for Diesel Engines
Getting Peer Group for Diversified
Getting Peer Group f

In [4]:
mojo_temp.drop_duplicates(inplace=True)
mojo_temp[(mojo_temp['Mojo'] == 'Strong Buy')].sort_values(by=['Industry','Name'])

,BSEID,NSEID,Name,Industry,quality,valuation,financial,technical,Mojo
424378,532768,FIEMIND,Fiem Industries,Auto Anc,Good,Very Attractive,Positive,Positive,Strong Buy
102348,541956,IRCON,Ircon International,Engineering,Good,Very Attractive,Positive,Positive,Strong Buy
892537,519183,ADFFOODS,ADF Foods,FMCG,Average,Very Attractive,Very Positive,Very Positive,Strong Buy
963994,500825,BRITANNIA,Britannia Industries,FMCG,Excellent,Expensive,Positive,Positive,Strong Buy
308453,532281,HCLTECH,HCL Technologies,IT - Software,Excellent,Attractive,Flat,Flat,Strong Buy
316432,532129,HEXAWARE,Hexaware Tech,IT - Software,Excellent,Attractive,Positive,Positive,Strong Buy
805539,531209,NUCLEUS,Nucleus Software Exp,IT - Software,Good,Attractive,Very Positive,Very Positive,Strong Buy
491198,506285,BAYERCROP,Bayer CropScience,Pesticides & Agrochem,Good,Very Expensive,Outstanding,Outstanding,Strong Buy
748382,507717,DHANUKA,Dhanuka Agritech,Pesticides & Agrochem,Good,Fair,Very Positive,Very Positive,Strong Buy
657348,524348,AARTIDRUGS,Aarti Drugs,Pharma,Average,Fair,Outstanding,Outstanding,Strong Buy


In [31]:
# Fetch the Stocks with Highest Potentials per Brokerage 
url = 'https://www.marketsmojo.com/recommendation/api/getRecos.php?List=hipot&sortColumn=POTEN&sortDirection=desc&page=1&pageSize=50&format=json'

s = requests.get(url)
df  = pd.DataFrame(s.json()['item'])[[ 'BrkName','' 'StkID','StkName','StkQlty','StkVal','StkCft','StkPrice','CallPrice','TargetPrice','CallStatus','CallSingnal','CallDate' ]]

In [32]:
pd.DataFrame(s.json()['item']).columns

Index(['CallID', 'BrkID', 'BrkName', 'BrkStrL', 'BrkRnkL', 'BrkStrM',
       'BrkRnkM', 'BrkStrS', 'BrkRnkS', 'StkID', 'StkName', 'StkEx', 'StkQlty',
       'StkVal', 'StkCft', 'StkCap', 'StkMCap', 'StkPrice', 'StkChg',
       'StkPchg', 'StkIndID', 'StkIndName', 'CallDirID', 'CallSingnal',
       'CallStatus', 'CallImg', 'CallCls', 'CallDate', 'CallPDate',
       'CallPrice', 'TargetPrice', 'Perfom', 'Potent_score', 'Potential',
       'Source', 'SortCol', 'Sourceurl'],
      dtype='object')

In [33]:
allgreen = (df.StkQlty=='green') & (df.StkVal=='green') & (df.StkCft=='green')

In [34]:
df[allgreen]

,BrkName,StkID,StkName,StkQlty,StkVal,StkCft,StkPrice,CallPrice,TargetPrice,CallStatus,CallSingnal,CallDate
2,Indiabulls Ventures,102348,Ircon International,green,green,green,96.40,511.30,615.00,1,Buy,13 Feb 20
47,HDFC Securities,1002932,HG Infra Engineering,green,green,green,181.85,239.86,441.00,1,Buy,23 Dec 19
